In [ ]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers

@route('/')
@route('/index')
@route('/index/')
def index():
    return template('index')

@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')

@route('/login/')
@route('/login')
def login():
    return template("login")

@route('/register')
@route('/register/')
def regiser():
    return template("register")

@route('/one_sample_ttest')
@route('/one_sample_ttest/')
def one_sample_ttest():
    return template("one_sample_ttest")

# Debug = True - changes in template are visible immidietly
run(host='localhost', port=8012, debug=True)

Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8012/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/Jan/2022 15:57:04] "GET / HTTP/1.1" 200 4283
127.0.0.1 - - [10/Jan/2022 15:57:05] "GET /one_sample_ttest HTTP/1.1" 200 6187
127.0.0.1 - - [10/Jan/2022 15:57:06] "GET /inedx HTTP/1.1" 404 730
127.0.0.1 - - [10/Jan/2022 15:57:20] "GET / HTTP/1.1" 200 4283
127.0.0.1 - - [10/Jan/2022 15:58:51] "GET /index HTTP/1.1" 200 4283
127.0.0.1 - - [10/Jan/2022 15:58:54] "GET /index HTTP/1.1" 200 4283
127.0.0.1 - - [10/Jan/2022 15:58:55] "GET /one_sample_ttest HTTP/1.1" 200 6187
127.0.0.1 - - [10/Jan/2022 15:58:56] "GET /inedx HTTP/1.1" 404 730
127.0.0.1 - - [10/Jan/2022 15:59:06] "GET /inedx HTTP/1.1" 404 730
127.0.0.1 - - [10/Jan/2022 15:59:08] "GET /index HTTP/1.1" 200 4283
127.0.0.1 - - [10/Jan/2022 15:59:12] "GET / HTTP/1.1" 200 4283
127.0.0.1 - - [10/Jan/2022 15:59:49] "GET / HTTP/1.1" 200 4283
127.0.0.1 - - [10/Jan/2022 15:59:49] "GET /static/css/styles.css H